# Closed-loop optimization using batch-sequential sampling

In [ ]:
# import all required packages 
import math
import numpy as np
import pandas as pd
from sklearn import metrics
from sklearn.utils import shuffle
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler

# for RF
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor

#for GP
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import RBF,Matern, ConstantKernel as C

# for the NN
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import tensorflow_docs as tfdocs
import tensorflow_docs.plots
import tensorflow_docs.modeling

import xlrd

import matplotlib.pyplot as plt

## Feature selection

In [ ]:
# Make the inputs and the outputs ready - the features can be generated using the separate file Preprocessing.ipynb

full_dataset = data_2_sterimol
full_dataset = np.append(full_dataset,dft_data,axis=1)
full_dataset = np.append(full_dataset,fp_pca_data,axis=1)
full_dataset = np.append(full_dataset,dataset_one_hot,axis=1)
full_dataset = np.append(full_dataset,NBO_data,axis=1)
full_dataset = np.append(full_dataset,chelpg_data,axis=1)
full_dataset = np.append(full_dataset,vol_bur,axis=1)
full_dataset = np.append(full_dataset,n_proton,axis=1)
full_dataset = np.append(full_dataset,yield_numerical,axis=1)

feature_column_names = ['N_sterimol_l','N_sterimol_b1','N_sterimol_b5','R_sterimol_l','R_sterimol_b1','R_sterimol_b5',
                       'homo', 'lumo',
                       'ligands_PCA_1',' ligands_PCA_2',' ligands_PCA_3','ligands_PCA_4','ligands_PCA_5','ligands_PCA_6','ligands_PCA_7','ligands_PCA_8','ligands_PCA_9','ligands_PCA_10','ligands_PCA_11','ligands_PCA_12','ligands_PCA_13','ligands_PCA_14','ligands_PCA_15','ligands_PCA_16','ligands_PCA_17','ligands_PCA_18','ligands_PCA_19','ligands_PCA_20','ligands_PCA_21','ligands_PCA_22','ligands_PCA_23','ligands_PCA_24','ligands_PCA_25','ligands_PCA_26','ligands_PCA_27','ligands_PCA_28','ligands_PCA_29','ligands_PCA_30',
                       'precat_PCA_1',' precat_PCA_2',' precat_PCA_3','precat_PCA_4','precat_PCA_5','precat_PCA_6','precat_PCA_7','precat_PCA_8','precat_PCA_9','precat_PCA_10','precat_PCA_11','precat_PCA_12','precat_PCA_13','precat_PCA_14','precat_PCA_15','precat_PCA_16','precat_PCA_17','precat_PCA_18','precat_PCA_19','precat_PCA_20','precat_PCA_21','precat_PCA_22','precat_PCA_23','precat_PCA_24','precat_PCA_25','precat_PCA_26','precat_PCA_27','precat_PCA_28','precat_PCA_29','precat_PCA_30',
                       'boronic_PCA_1',' boronic_PCA_2',' boronic_PCA_3','boronic_PCA_4','boronic_PCA_5','boronic_PCA_6','boronic_PCA_7','boronic_PCA_8','boronic_PCA_9','boronic_PCA_10','boronic_PCA_11','boronic_PCA_12','boronic_PCA_13','boronic_PCA_14','boronic_PCA_15','boronic_PCA_16','boronic_PCA_17','boronic_PCA_18','boronic_PCA_19','boronic_PCA_20','boronic_PCA_21','boronic_PCA_22','boronic_PCA_23','boronic_PCA_24','boronic_PCA_25','boronic_PCA_26','boronic_PCA_27','boronic_PCA_28','boronic_PCA_29','boronic_PCA_30',
                       'OHE_lig_1',' OHE_lig_2',' OHE_lig_3','OHE_lig_4','OHE_lig_5','OHE_lig_6','OHE_lig_7','OHE_lig_9','OHE_lig_10','OHE_lig_11','OHE_lig_12','OHE_lig_13','OHE_lig_14','OHE_lig_15','OHE_lig_16','OHE_lig_17','OHE_lig_18','OHE_lig_19','OHE_lig_20','OHE_lig_21','OHE_lig_22','OHE_lig_23','OHE_lig_24','OHE_lig_25','OHE_lig_26','OHE_lig_27','OHE_lig_28','OHE_lig_29','OHE_lig_30',' OHE_lig_31',' OHE_lig_32',
                       'OHE_precat_1','OHE_precat_2','OHE_precat_3',
                       'OHE_bor_1','OHE_bor_2',
                       'NBO_N','NBO_OR','NBO_O','NBO_AcO',
                       'chelpg_N','chelpg_OR','chelpg_O','chelpg_AcO',
                       'vol_bur',
                       'n_proton',
                       'Yield']

full_dataset_input = pd.DataFrame(full_dataset)
full_dataset_input.columns = feature_column_names

print('Shape of the featurized dataset:',full_dataset_input.shape)

#### To allow for fair comparison across different models when conducting 10 repetitions, the data used for initialization was set a priori for various initialization strategies

In [ ]:
# 10 random initializations
random_initialization = [162, 129,  89, 40,124,  16, 165, 100, 177, 108, 138, 104, 103,  79, 152, 137, 112,44, 124, 175, 164, 149,   9,  36]
#random_initialization = [161,  98, 147, 158, 118, 180, 184,  93,  57,  31, 136, 103, 151, 112, 162,  57,   1,  74,  62, 112,   7,  55,  99,   9]
#random_initialization = [155,  23, 169,   2,  80,  56, 179, 139,  73,  42,  41,  83, 168, 64, 155,  23,  54, 171,  42, 166, 122, 106,   8, 147]
#random_initialization = [ 68,  54, 163, 175,  62, 108,  85,  24, 180, 116, 123,  25, 149, 87,  79,   1,  47, 104, 113,  69,   0, 175,  85,  79]
#random_initialization = [ 52, 127,  89,  14,  64,   3,  41,  55,  84, 165,   0,  28,  30, 20, 124,  72,  52,  68,   9,  49, 184, 139, 127, 138]
#random_initialization = [ 95,   9,  86,  82, 177,  85, 183, 133,  63, 155,  76,   6, 173, 71,  15, 179,  51,  28,  10, 102,  57,  73,   1, 129]
#random_initialization = [ 95, 183, 118, 110,  61,  24,  82, 143, 150,  37, 138, 166,  18, 131,  35,  34, 159,  23, 139,  68,  22,  18, 153, 120]
#random_initialization = [ 33,  82, 161,  49,  12, 116,  40,  87,  50, 164, 177,  37, 126, 166,  32, 127, 180,  68,  74, 134, 103,  35,   9,  24]
#random_initialization = [131,  78,  13,  81, 139,  30,  79,  25,  29,  73,  46,  50,  19, 60, 151, 108, 121, 134,  72,  79,  43, 161, 132,  97]
#random_initialization = [ 22,   2, 129,  68, 161,   7,  63,  24,  85,  18,  92, 162, 175, 45, 153,   0,  67,  73,  92,  69, 153,  61,  10, 149]

initial_data = full_dataset_input.loc[random_initialization]

lookup_table = full_dataset_input.drop(random_initialization)

lookup_table = shuffle(lookup_table)

## Let's illustrate the data for initialization on the  chemical space

In [ ]:
pca_illustrate = PCA(n_components=2)
fingerprints_pca_2 = pca_illustrate.fit_transform(full_dataset_input)
print(fingerprints_pca_2.shape)
fingerprints_pca_2 = np.array(fingerprints_pca_2)

fingerprints_init_pca = fingerprints_pca_2[random_initialization]
print(fingerprints_init_pca.shape)
#print(fingerprints_init_pca)

fingerprints_lookupTable_pca = np.delete(fingerprints_pca_2, random_initialization, 0)
print(fingerprints_lookupTable_pca.shape)

plt.title(label="Location of the initialization data on the chemical space")

plt.scatter(fingerprints_lookupTable_pca[:,0], fingerprints_lookupTable_pca[:,1],label='Lookup_data',color='dodgerblue')
plt.scatter(fingerprints_init_pca[:6,0],fingerprints_init_pca[:6,1] ,label='Initial data',color='fuchsia')
plt.xlabel('PCA1')
plt.ylabel('PCA2')
plt.legend()

### Train the initial random forest model

In [ ]:
iniit_output = initial_data['Yield']
print(iniit_output.shape)

iniit_input = initial_data.drop(columns='Yield')
print(iniit_input.shape)

train_features_init = iniit_input[14:]
train_labels_init = iniit_output[14:]
test_features_init = iniit_input[:14]
test_labels_init = iniit_output[:14]

regressor = RandomForestRegressor(n_estimators=200, random_state=0)
regressor.fit(train_features_init, train_labels_init)
test_predictions_init = regressor.predict(test_features_init)

a = plt.axes(aspect='equal')
plt.scatter(test_labels_init, test_predictions_init)
plt.xlabel('True Values')
plt.ylabel('Predictions')
lims = [1e-04,1.2]
plt.xlim(lims)
plt.ylim(lims)
_ = plt.plot(lims, lims)

lookup_output = lookup_table['Yield']
print(lookup_output.shape)

lookup_input = lookup_table.drop(columns='Yield')
print(lookup_input.shape)

### Starting the active learning

In [ ]:
dataset_shape = []
RMSE = []
lookup_input = np.array(lookup_input)
lookup_output = np.array(lookup_output)
lookup_output = lookup_output.reshape(-1,1)
pred_yield_development = []

# set the very initial train features and labels
train_features = train_features_init
train_labels = train_labels_init

train_labels = np.array(train_labels).reshape(-1,1)

current_maximum =[]

#set back to 148 when done

for i in range (6):
    train_x = train_features 
    train_y = train_labels
    
    # let's train the RF
    regressor = RandomForestRegressor(n_estimators=200, random_state=0)
    regressor.fit(train_x, train_y)
    test_predictions = regressor.predict(test_features_init)
    
    current_maximum.append(np.max(train_labels))
    
    ''' inner loop for batch sampling '''
    
    next_batch = []
    
    single_yields = []
    
    lookup_input_temp = lookup_input
    lookup_output_temp = lookup_output
    
    for x in range (25):
        
        lookup_predictions = regressor.predict(lookup_input_temp)
        best = np.max(lookup_predictions)
        List = lookup_predictions.tolist()
        index = List.index(best)
        next_batch.append(index)
                
        lookup_input_temp = np.delete(lookup_input_temp, index, 0)
        lookup_output_temp = np.delete(lookup_output_temp, index, 0)
        
        single_yields.append(best)
        
        
    new_input = lookup_input[next_batch]
    new_output = lookup_output[next_batch]
    
    train_features = np.append(train_features,new_input,axis=0)
    train_labels = np.append(train_labels,new_output,axis=0)

    lookup_input = np.delete(lookup_input, [next_batch], 0)
    lookup_output = np.delete(lookup_output, [next_batch], 0)
    
    pred_yield_development.append(np.mean(single_yields))
    
    print('Batch:',i, '  the RMSE is:',rmse, '   average yield:',np.mean(single_yields))

### Evaluation

In [ ]:
plt.plot(RMSE)
plt.xlabel('Number of iterations')
plt.ylabel('RMSE')
plt.title(label="Optimistic active learning")
plt.show()

In [ ]:
plt.plot(pred_yield_development)
plt.xlabel('Number of Batch')
plt.ylabel('Average Batch Yield')
plt.title(label="Development of predicted yield over multiple iterations")
plt.show()

In [ ]:
plt.plot(current_maximum)
plt.xlabel('Number of Batch')
plt.ylabel('Best yield')
plt.title(label="Development of maximal observed yield")
plt.show()

print(current_maximum)